In [4]:
# Import dependencies
import twitter
from bs4 import BeautifulSoup
import requests
import pymongo
import pprint
from datetime import datetime, timedelta
# Import program used to categorize sentiments
import sent_rating 
import config as config

In [5]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [6]:
# Define database and collection
db = client.twitter_db
collection = db.tweets
collection.drop()

In [7]:
api = twitter.Api(consumer_key=config.API_KEY,
  consumer_secret=config.API_SECRET,
  access_token_key=config.ACCESS_TOKEN,
  access_token_secret=config.ACCESS_TOKEN_SECRET)

In [8]:
print(api.VerifyCredentials())

{"created_at": "Sun Feb 23 18:39:51 +0000 2014", "default_profile": true, "default_profile_image": true, "followers_count": 2, "friends_count": 9, "id": 2358358214, "id_str": "2358358214", "name": "Sanjeev", "profile_background_color": "C0DEED", "profile_background_image_url": "http://abs.twimg.com/images/themes/theme1/bg.png", "profile_background_image_url_https": "https://abs.twimg.com/images/themes/theme1/bg.png", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "sanmankar", "status": {"created_at": "Mon Feb 02 02:04:54 +0000 2015", "id": 562069115272454144, "id_str": "562069115272454144", "lang": "de", "source": "<a href=\"http

# Twitter API Call syntax

### GetSearch(term=None, raw_query=None, geocode=None, since_id=None, max_id=None, until=None, since=None, count=15, lang=None, locale=None, result_type='mixed', include_entities=None, return_json=False)

In [9]:
##### TWITTER SEARCH EXAMPLE and CODE to test OUT ########

v_startdate = "2019-06-10"
# Calculate the date ranges to pick up tweets. Twitter does not allow fetching tweets older than a week.
N=11
v_rng_end_dt = datetime.today().strftime('%Y-%m-%d')
v_rng_start_dt = (datetime.today() - timedelta(days=N)).strftime('%Y-%m-%d')

#v_enddate = "2019-06-03"
v_lang = "en"
v_count = 1
v_geo_code = [40.7128, -74.0060, "5000mi"]
v_search_str = 'dunkin coffee'

v_enddate = (datetime.strptime(v_startdate, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')

#print(v_enddate)
pp = pprint.PrettyPrinter(indent=4)
search = api.GetSearch(term=v_search_str,since=v_startdate,until=v_enddate,lang=v_lang,count=v_count)
#pp.pprint(search);
#print(search)

for tweet in search:
#     print(f'ID: {tweet.id} CREATED: {tweet.created_at} FAV COUNT: {tweet.user.favourites_count}')
    #print(tweet.user.favourites_count)
    print(tweet)


{"created_at": "Mon Jun 10 13:03:43 +0000 2019", "favorite_count": 223, "hashtags": [], "id": 1138069228450459648, "id_str": "1138069228450459648", "lang": "en", "retweet_count": 17, "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>", "text": "Haven\u2019t been on the Acela for a while. Relieved to hear announcement that it continues to \u201cproudly serve freshly br\u2026 https://t.co/oZ4HnCGjB0", "truncated": true, "urls": [{"expanded_url": "https://twitter.com/i/web/status/1138069228450459648", "url": "https://t.co/oZ4HnCGjB0"}], "user": {"created_at": "Thu Oct 02 18:47:57 +0000 2014", "default_profile": true, "description": "Director, https://t.co/PWRcR3IxJj, https://t.co/Cxyk3n1K1G. Host, https://t.co/2cRflRmz1y.", "followers_count": 482567, "friends_count": 4082, "id": 2800581040, "id_str": "2800581040", "lang": "en", "listed_count": 5647, "name": "Bill Kristol", "profile_background_color": "C0DEED", "profile_background_image_url": 

In [10]:
# Create a search list for companies
search_list = [{"id":"SB","name":"Star Bucks","search_str":'Starbucks Coffee'},
               {"id":"MD","name":"McDonalds","search_str":"McDonald Coffee"},
              {"id":"DD","name":"Dunkin Donuts","search_str":"Dunkin Coffee"}]


# Calculate a date range array with a range starting from current date going  back 10 days
N=10
v_curr_date = datetime.today().strftime('%Y-%m-%d')
v_rng_strt_dt_char =  (datetime.today() - timedelta(days=N)).strftime('%Y-%m-%d')
v_rng_strt_dt = datetime.strptime(v_rng_strt_dt_char,'%Y-%m-%d')
date_arr=[]
for i in range(0,11):
   date_arr.append((v_rng_strt_dt + timedelta(days=i)).strftime('%Y-%m-%d'))

v_lang = "en"
v_count = 1000
v_geo_code = [40.7128, -74.0060, "5000mi"]
v_sentiment=""

for x in search_list:

  for v_strt_dt in date_arr:
    
        query = x['search_str']
        v_id = x['id']
        v_name = x['name']
        
        v_end_dt = (datetime.strptime(v_strt_dt, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
    
        search = api.GetSearch(term=query,since=v_strt_dt,until=v_end_dt,lang=v_lang,count=v_count)


        for tweet in search:
            # Error handling
            try:
        
                # Print the output from twitter on the screen
                #print(tweet.id, tweet.text);
                #create_date = datetime.strptime(tweet.created_at, '%a %b %d %H:%M:%S +%f %Y').strftime('%d/%m/%Y')
                create_date = datetime.strptime(tweet.created_at, '%a %b %d %H:%M:%S +%f %Y').strftime('%Y-%m-%d')
                print(f'Create Date: {create_date} Rng_Str: {v_strt_dt} Rng_End: {v_end_dt}')
            
                v_sentiment = sent_rating.get_rating(tweet.text);
                
              
            
                # Dictionary to be inserted as a MongoDB document
                post = {
                    'id': v_id,
                    'name': v_name,
                    'created' : create_date,
                    'range_strt_dt': v_strt_dt,
                    'range_end_dt' : v_end_dt,
                    'search_str': query,
                    'tweet_id': tweet.id,
                    'tweet_text': tweet.text,
                    'retweet_count': tweet.retweet_count,
                    'source' : tweet.source,
                    'favourite_count' : tweet.user.favourites_count,
                    'sentiment' : v_sentiment
                }

                collection.insert_one(post)

            except Exception as e:
                print(e)
        #print(tweet.id);

Create Date: 2019-06-05 Rng_Str: 2019-06-05 Rng_End: 2019-06-06
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-05 Rng_Str: 2019-06-05 Rng_End: 2019-06-06
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 1 Negative : 1
Create Date: 2019-06-05 Rng_Str: 2019-06-05 Rng_End: 2019-06-06
Number of Negative words : 4783
Number of Positive words : 2005
Negative Tweet : 2
Create Date: 2019-06-05 Rng_Str: 2019-06-05 Rng_End: 2019-06-06
Number of Negative words : 4783
Number of Positive words : 2005
Positive Tweet : 1
Create Date: 2019-06-05 Rng_Str: 2019-06-05 Rng_End: 2019-06-06
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-05 Rng_Str: 2019-06-05 Rng_End: 2019-06-06
Number of Negative words : 4783
Number of Positive words : 2005
Negative Tweet : 2
Create Date: 2019-06-05 Rng_Str: 2019-06-05 Rn

Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-05 Rng_Str: 2019-06-05 Rng_End: 2019-06-06
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-05 Rng_Str: 2019-06-05 Rng_End: 2019-06-06
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-05 Rng_Str: 2019-06-05 Rng_End: 2019-06-06
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-05 Rng_Str: 2019-06-05 Rng_End: 2019-06-06
Number of Negative words : 4783
Number of Positive words : 2005
Positive Tweet : 1
Create Date: 2019-06-05 Rng_Str: 2019-06-05 Rng_End: 2019-06-06
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-05 Rng_Str: 2019-06-05 Rng_End: 2019-06-0

Number of Negative words : 4783
Number of Positive words : 2005
Positive Tweet : 2
Create Date: 2019-06-06 Rng_Str: 2019-06-06 Rng_End: 2019-06-07
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-06 Rng_Str: 2019-06-06 Rng_End: 2019-06-07
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-06 Rng_Str: 2019-06-06 Rng_End: 2019-06-07
Number of Negative words : 4783
Number of Positive words : 2005
Positive Tweet : 2
Create Date: 2019-06-06 Rng_Str: 2019-06-06 Rng_End: 2019-06-07
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-06 Rng_Str: 2019-06-06 Rng_End: 2019-06-07
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 1 Negative : 1
Create Date: 2019-06-06 Rng_Str: 2019-06-06 Rng_End: 2019-06-07
Number of Negative wor

Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-06 Rng_Str: 2019-06-06 Rng_End: 2019-06-07
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-06 Rng_Str: 2019-06-06 Rng_End: 2019-06-07
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-06 Rng_Str: 2019-06-06 Rng_End: 2019-06-07
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 1 Negative : 1
Create Date: 2019-06-06 Rng_Str: 2019-06-06 Rng_End: 2019-06-07
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-06 Rng_Str: 2019-06-06 Rng_End: 2019-06-07
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-06 Rng_Str: 2019-

Number of Positive words : 2005
Neutral Tweet :  Positive : 1 Negative : 1
Create Date: 2019-06-07 Rng_Str: 2019-06-07 Rng_End: 2019-06-08
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-07 Rng_Str: 2019-06-07 Rng_End: 2019-06-08
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-07 Rng_Str: 2019-06-07 Rng_End: 2019-06-08
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-07 Rng_Str: 2019-06-07 Rng_End: 2019-06-08
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-07 Rng_Str: 2019-06-07 Rng_End: 2019-06-08
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-07 Rng_Str: 2019-06-07 Rng_End: 2019-06-08
Number

...

Create Date: 2019-06-13 Rng_Str: 2019-06-13 Rng_End: 2019-06-14
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 1 Negative : 1
Create Date: 2019-06-13 Rng_Str: 2019-06-13 Rng_End: 2019-06-14
Number of Negative words : 4783
Number of Positive words : 2005
Negative Tweet : 1
Create Date: 2019-06-13 Rng_Str: 2019-06-13 Rng_End: 2019-06-14
Number of Negative words : 4783
Number of Positive words : 2005
Negative Tweet : 2
Create Date: 2019-06-13 Rng_Str: 2019-06-13 Rng_End: 2019-06-14
Number of Negative words : 4783
Number of Positive words : 2005
Negative Tweet : 1
Create Date: 2019-06-13 Rng_Str: 2019-06-13 Rng_End: 2019-06-14
Number of Negative words : 4783
Number of Positive words : 2005
Positive Tweet : 1
Create Date: 2019-06-13 Rng_Str: 2019-06-13 Rng_End: 2019-06-14
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-13 Rng_Str: 2019-06-13 Rng_End: 2019-06-14
Number

Create Date: 2019-06-13 Rng_Str: 2019-06-13 Rng_End: 2019-06-14
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-13 Rng_Str: 2019-06-13 Rng_End: 2019-06-14
Number of Negative words : 4783
Number of Positive words : 2005
Positive Tweet : 2
Create Date: 2019-06-13 Rng_Str: 2019-06-13 Rng_End: 2019-06-14
Number of Negative words : 4783
Number of Positive words : 2005
Negative Tweet : 1
Create Date: 2019-06-13 Rng_Str: 2019-06-13 Rng_End: 2019-06-14
Number of Negative words : 4783
Number of Positive words : 2005
Positive Tweet : 2
Create Date: 2019-06-13 Rng_Str: 2019-06-13 Rng_End: 2019-06-14
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-13 Rng_Str: 2019-06-13 Rng_End: 2019-06-14
Number of Negative words : 4783
Number of Positive words : 2005
Negative Tweet : 1
Create Date: 2019-06-13 Rng_Str: 2019-06-13 Rng_End: 2019-06-14
Number

Number of Negative words : 4783
Number of Positive words : 2005
Positive Tweet : 1
Create Date: 2019-06-14 Rng_Str: 2019-06-14 Rng_End: 2019-06-15
Number of Negative words : 4783
Number of Positive words : 2005
Negative Tweet : 1
Create Date: 2019-06-14 Rng_Str: 2019-06-14 Rng_End: 2019-06-15
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-14 Rng_Str: 2019-06-14 Rng_End: 2019-06-15
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 1 Negative : 1
Create Date: 2019-06-14 Rng_Str: 2019-06-14 Rng_End: 2019-06-15
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-14 Rng_Str: 2019-06-14 Rng_End: 2019-06-15
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-14 Rng_Str: 2019-06-14 Rng_End: 2019-06-15
Number of Negative wor

Create Date: 2019-06-15 Rng_Str: 2019-06-15 Rng_End: 2019-06-16
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-15 Rng_Str: 2019-06-15 Rng_End: 2019-06-16
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-15 Rng_Str: 2019-06-15 Rng_End: 2019-06-16
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-15 Rng_Str: 2019-06-15 Rng_End: 2019-06-16
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-15 Rng_Str: 2019-06-15 Rng_End: 2019-06-16
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 1 Negative : 1
Create Date: 2019-06-15 Rng_Str: 2019-06-15 Rng_End: 2019-06-16
Number of Negative words : 4783
Number of Positive words : 2005
Positive Tweet : 

Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-15 Rng_Str: 2019-06-15 Rng_End: 2019-06-16
Number of Negative words : 4783
Number of Positive words : 2005
Negative Tweet : 3
Create Date: 2019-06-15 Rng_Str: 2019-06-15 Rng_End: 2019-06-16
Number of Negative words : 4783
Number of Positive words : 2005
Neutral Tweet :  Positive : 0 Negative : 0
Create Date: 2019-06-15 Rng_Str: 2019-06-15 Rng_End: 2019-06-16
Number of Negative words : 4783
Number of Positive words : 2005
Positive Tweet : 1
Create Date: 2019-06-15 Rng_Str: 2019-06-15 Rng_End: 2019-06-16
Number of Negative words : 4783
Number of Positive words : 2005
Negative Tweet : 2
Create Date: 2019-06-15 Rng_Str: 2019-06-15 Rng_End: 2019-06-16
Number of Negative words : 4783
Number of Positive words : 2005
Negative Tweet : 2
Create Date: 2019-06-15 Rng_Str: 2019-06-15 Rng_End: 2019-06-16
Number of Negative words : 4783
Number of Positive words : 2005
Positi

In [11]:
# Display items in MongoDB collection
tweet_recs = db.tweets.find()

for tweet in tweet_recs:
    print(tweet)

{'_id': ObjectId('5d04fe685e8c23063839db70'), 'id': 'SB', 'name': 'Star Bucks', 'created': '2019-06-05', 'range_strt_dt': '2019-06-05', 'range_end_dt': '2019-06-06', 'search_str': 'Starbucks Coffee', 'tweet_id': 1136422181397831681, 'tweet_text': 'crossfaded off this starbucks coffee and this starbucks juice', 'retweet_count': 0, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'favourite_count': 51898, 'sentiment': 'Neutral'}
{'_id': ObjectId('5d04fe685e8c23063839db71'), 'id': 'SB', 'name': 'Star Bucks', 'created': '2019-06-05', 'range_strt_dt': '2019-06-05', 'range_end_dt': '2019-06-06', 'search_str': 'Starbucks Coffee', 'tweet_id': 1136422178143002624, 'tweet_text': '@ForcePao 2.50? That’s a good deal compared to the loan I have to get every time I get a crappy burnt coffee at Starbucks.☺️💸💸💸', 'retweet_count': 0, 'source': '<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>', 'favourite_count': 7599, 'sen

{'_id': ObjectId('5d04fe775e8c23063839e0e6'), 'id': 'MD', 'name': 'McDonalds', 'created': '2019-06-08', 'range_strt_dt': '2019-06-08', 'range_end_dt': '2019-06-09', 'search_str': 'McDonald Coffee', 'tweet_id': 1137243533973360640, 'tweet_text': 'If anyone wants to get me an iced coffee from McDonald’s, that’d be AMAZING', 'retweet_count': 0, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'favourite_count': 840, 'sentiment': 'Positive'}
{'_id': ObjectId('5d04fe775e8c23063839e0e7'), 'id': 'MD', 'name': 'McDonalds', 'created': '2019-06-08', 'range_strt_dt': '2019-06-08', 'range_end_dt': '2019-06-09', 'search_str': 'McDonald Coffee', 'tweet_id': 1137242609720745987, 'tweet_text': "PSA: McDonald's Now Has Coffee Ice Cream And It Is Legit https://t.co/hPgtULaUF1", 'retweet_count': 0, 'source': '<a href="http://www.facebook.com/twitter" rel="nofollow">Facebook</a>', 'favourite_count': 16, 'sentiment': 'Neutral'}
{'_id': ObjectId('5d04fe775e8c23

{'_id': ObjectId('5d04fe835e8c23063839e4cd'), 'id': 'DD', 'name': 'Dunkin Donuts', 'created': '2019-06-08', 'range_strt_dt': '2019-06-08', 'range_end_dt': '2019-06-09', 'search_str': 'Dunkin Coffee', 'tweet_id': 1137441441372934150, 'tweet_text': "@wetbuckets88 McDonald's coffee &gt;&gt; Dunkin coffee", 'retweet_count': 1, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'favourite_count': 31161, 'sentiment': 'Neutral'}
{'_id': ObjectId('5d04fe835e8c23063839e4ce'), 'id': 'DD', 'name': 'Dunkin Donuts', 'created': '2019-06-08', 'range_strt_dt': '2019-06-08', 'range_end_dt': '2019-06-09', 'search_str': 'Dunkin Coffee', 'tweet_id': 1137440080061259777, 'tweet_text': 'Sneaking my Dunkin iced coffee into the movie theater. Living life on the edge.', 'retweet_count': 0, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'favourite_count': 1929, 'sentiment': 'Neutral'}
{'_id': ObjectId('5d04fe835e8c

In [10]:
db.tweets.count() 

/Users/sanmankar/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


2986

In [12]:
y=db.tweets.aggregate([
{ "$group": { "_id": "$name", "No_of_Times": { "$sum": 1 } } }
])

for t in y:
    print(t)

{'_id': 'McDonalds', 'No_of_Times': 969}
{'_id': 'Dunkin Donuts', 'No_of_Times': 1044}
{'_id': 'Star Bucks', 'No_of_Times': 1100}


In [15]:
y=db.tweets.aggregate([
{ "$group": { "_id": "$sentiment", "No_of_Times": { "$sum": "$retweet_count" } } }
])

for t in y:
    print(t)

{'_id': 'Positive', 'No_of_Times': 879629}
{'_id': 'Negative', 'No_of_Times': 24005}
{'_id': 'Neutral', 'No_of_Times': 1704538}


In [42]:
metadata_rec = db.tweets.find( { 'id': "SB" } ).sort([("created", -1)]).limit(3)

sample_metadata={}
i=1
for tweet in metadata_rec:  
        #print(tweet)
        tim=tweet['created']        
        sample_metadata[i] = tweet['created'] + ' ::: ' + tweet['tweet_text']
        i = i + 1;

print(sample_metadata)


{1: '2019-06-15 ::: When you put milk in Starbucks coffee it doesn’t change the color one bit', 2: '2019-06-15 ::: RT @kkkkaatia: next time you go to starbucks don’t use a plastic cup just have them pour the coffee right into your car! 😌✨ https://t.co/7h…', 3: '2019-06-15 ::: RT @trashygaytweets: Me walking in to Starbucks and seeing that the barista at the register is gay, the barista making the drinks is gay, a…'}


In [12]:
x = "Tue Jun 04 23:42:11 +0000 2019"
datetime.strptime(x, '%a %b %d %H:%M:%S +%f %Y').strftime('%d/%m/%Y')

'04/06/2019'

# APPENDIX

In [ ]:
#search = api.GetSearch("MCDonald coffee") # Replace happy with your search

# Create a search list for companies
search_list = [{"id":"SB","name":"Star Bucks","search_str":'Starbucks Coffee'},
               {"id":"MD","name":"McDonalds","search_str":"McDonald Coffee"},
              {"id":"DD","name":"Dunkin Donuts","search_str":"Dunkin Coffee"}]

# Calculate a date range array with a range starting from current date going  back 10 days
N=11
v_curr_date = datetime.today().strftime('%Y-%m-%d')
v_rng_strt_dt_char =  (datetime.today() - timedelta(days=N)).strftime('%Y-%m-%d')
v_rng_strt_dt = datetime.strptime(v_rng_strt_dt,'%Y-%m-%d')
date_arr=[]
for i in range(0,11):
   date_arr.append((v_rng_strt_dt + timedelta(days=i)).strftime('%Y-%m-%d'))

print(date_arr)


## Static values
#v_startdate = ["2019-06-02","2019-06-03","2019-06-04","2019-06-05","2019-06-06","2019-06-07","2019-06-08",
#               "2019-06-09","2019-06-10","2019-06-11","2019-06-12"]

#v_enddate = "2019-06-10"
v_lang = "en"
v_count = 1000
v_geo_code = [40.7128, -74.0060, "5000mi"]
v_sentiment=""

# Calculate the end date by incrementing one day
#v_enddate = (datetime.strptime(v_startdate, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')


for x in search_list:

  for v_strt_dt in v_startdate:
    
        query = x['search_str']
        v_id = x['id']
        v_name = x['name']
        
        v_end_dt = (datetime.strptime(v_strt_dt, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
    
        search = api.GetSearch(term=query,since=v_strt_dt,until=v_end_dt,lang=v_lang,count=v_count)


        for tweet in search:
            # Error handling
            try:
        
                # Print the output from twitter on the screen
                print(tweet.id, tweet.text);
            
                v_sentiment = sent_rating.get_rating(tweet.text);
            
                # Dictionary to be inserted as a MongoDB document
                post = {
                    'id': v_id,
                    'name': v_name,
                    'created' : tweet.created_at,
                    'search_str': query,
                    'tweet_id': tweet.id,
                    'tweet_text': tweet.text,
                    'favourite_count' : tweet.user.favourites_count,
                    'sentiment' : v_sentiment
                }

                collection.insert_one(post)

            except Exception as e:
                print(e)
        #print(tweet.id);